In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install unsloth

In [3]:
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

Found existing installation: unsloth 2025.3.1
Uninstalling unsloth-2025.3.1:
  Successfully uninstalled unsloth-2025.3.1
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-a06ewl4o
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-a06ewl4o
  Resolved https://github.com/unslothai/unsloth.git to commit 5d0ee525c1b6a3522f64ad9722249ae34b584555
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.3.1-py3-none-any.whl size=189066 sha256=e9debcd658bd813b66c6c96d8a68a851234883f92a420f2bcdc1d342ddbbec5a
  Stored in directory: /tmp/pip-ephem-wheel-cache-3ea2q48m/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth


In [ ]:
import torch
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from transformers import AutoTokenizer, TrainingArguments
from trl import SFTTrainer

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.1: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
text = "တောင်တောရယ်သာ မာလာက ငုံဖူး"
encoded_text = tokenizer.encode(text)
decoded_text = tokenizer.decode(encoded_text)
print(f"Encoded : {encoded_text}")
print(f"Decoded : {decoded_text}")

Encoded : [128000, 25870, 238, 25870, 109, 25870, 105, 25870, 226, 25870, 118, 25870, 238, 25870, 109, 25870, 105, 25870, 249, 25870, 248, 25870, 118, 25870, 252, 25870, 105, 220, 25870, 247, 25870, 105, 25870, 250, 25870, 105, 25870, 222, 220, 25870, 226, 25870, 107, 25870, 114, 25870, 244, 25870, 108, 25870, 116]
Decoded : <｜begin▁of▁sentence｜>တောင်တောရယ်သာ မာလာက ငုံဖူး


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2025.3.1 patched 32 layers with 32 QKV layers, 32 O layers and 0 MLP layers.


In [ ]:
dataset = load_dataset('json', data_files="/content/drive/MyDrive/R1_8B_Unsloth/Microbiology.json", split = "train")
print(dataset.column_names)
dataset

['Instruction', 'Input', 'Output']


Dataset({
    features: ['Instruction', 'Input', 'Output'],
    num_rows: 1260
})

In [ ]:
## Define your chat template
chat_template = """
### Instruction:
{INPUT}

### Response:
{OUTPUT}"""

## Apply the chat template manually
def apply_chat_template_manually(example, chat_template):
    formatted_text = chat_template.format(INPUT=example["Instruction"], OUTPUT= example["Output"])
    return {"text": formatted_text}

dataset = dataset.map(apply_chat_template_manually, fn_kwargs={"chat_template": chat_template})
dataset

Map:   0%|          | 0/1260 [00:00<?, ? examples/s]

Dataset({
    features: ['Instruction', 'Input', 'Output', 'text'],
    num_rows: 1260
})

In [41]:
dataset[0]

{'Instruction': 'အဏုဇီဝဗေဒ ဆိုတာ ဘာလဲ ရှင်းပြပါ',
 'Input': '',
 'Output': 'အဏုဇီဝဗေဒသည် မျက်စိဖြင့် မမြင်နိုင်သော အဏုဇီဝ သက်ရှိများကို လေ့လာသည့် သိပ္ပံပညာရပ် ဖြစ်သည် ။ ဤပညာရပ်တွင် ဘက်တီးရီးယား ၊ ဗိုင်းရပ်စ်၊ မှို၊ နှင့် protozoa များကို အဓိက လေ့လာသည် ။',
 'text': '\n### Instruction:\nအဏုဇီဝဗေဒ ဆိုတာ ဘာလဲ ရှင်းပြပါ\n\n### Response:\nအဏုဇီဝဗေဒသည် မျက်စိဖြင့် မမြင်နိုင်သော အဏုဇီဝ သက်ရှိများကို လေ့လာသည့် သိပ္ပံပညာရပ် ဖြစ်သည် ။ ဤပညာရပ်တွင် ဘက်တီးရီးယား ၊ ဗိုင်းရပ်စ်၊ မှို၊ နှင့် protozoa များကို အဓိက လေ့လာသည် ။'}

Total batch size = 16:
The effective batch size is 4 (batch size) * 4 (gradient accumulation) = 16

Steps per epoch
=  Num examples / Total batch size
​= 1,260 / 16
​= 78.75 ≈ 79 steps

In [ ]:
## Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=2000,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="/content/drive/MyDrive/R1_8B_Unsloth/outputs",
        report_to="none",  # Use this for WandB etc
    ),
)
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,260 | Num Epochs = 26
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 2,000
 "-____-"     Number of trainable parameters = 3,407,872


Step,Training Loss
10,0.541600
20,0.559500
30,0.494500
40,0.474400
50,0.446200
60,0.437000
70,0.431600
80,0.408700
90,0.364100
100,0.355900


In [ ]:
## Save the model and tokenizer
model.save_pretrained("/content/drive/MyDrive/R1_8B_Unsloth/outputs/model")
tokenizer.save_pretrained("/content/drive/MyDrive/R1_8B_Unsloth/outputs/model")

### Inference

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

## Load the model and tokenizer
model2, tokenizer2 = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/R1_8B_Unsloth/outputs/model",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

## Prepare the model for inference
model2 = FastLanguageModel.for_inference(model2)

==((====))==  Unsloth 2025.3.1: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
## Define a function for inference
def generate_response(model, tokenizer, input_text, max_length=2048):
    ## Encode the input text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    ## Generate the output
    outputs = model.generate(
        input_ids=inputs["input_ids"].to(model.device),
        attention_mask=inputs["attention_mask"].to(model.device),
        max_length=max_length,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
    )

    ## Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

input_text = "Capsule ၏ လုပ်ဆောင်ချက်ကို ရှင်းပြပါ ။"
response = generate_response(model2, tokenizer2, input_text)
print(f"Response: {response}")

Response: Capsule ၏ လုပ်ဆောင်ချက်ကို ရှင်းပြပါ ။

### လုပ်ဆောင်ချက်များ

1. **ကွန်ပွဲခြင်း**:
   - အသွေအချိုးအမျိုးဖြင့် ကွန်ပွဲခြင်း ရှိပါ လည်း လုပ်ဆောင်ချက်ကို ဆောင်သည် အချိုးပိုမိုးရှိသော ကွန်ပွဲများနှင့် အမျိုးရှိသော ကွန်ပွဲများ ဆောင်ချသည် ။

2. **စက်ရှင်းခြင်း**:
   - ကွန်ပွဲများက စက်ရှင်းခြင်းများကို ဆောင်ချသည် ။ စက်ရှင်းခြင်းများက အသွေအချိုးများရှိသော ကွန်ပွဲများ များစွာ ဆောင်ချသည် ။

3. **အသွေအချိုး များစွာ ဆောင်ချခြင်း**:
   - အသွေအချိုး များစွာ ဆောင်ချသည် ။ အသွေအချိုး များစွာ ရှိသော ကွန်ပွဲများ ဆောင်ချသည် ။

4. **အချိုးဆောင်ခြင်း**:
   - အချိုးဆောင်ခြင်း ရှိသော ကွန်ပွဲများ ဆောင်ချသည် ။ အချိုးဆောင်ခြင်းများက အသွေအချိုး များစွာ ဆောင်ချသည် ။

### လုပ်ဆောင်ချက်များ

1. **အသွေအချိုးရှိ ကွန်ပွဲများ**:
   - အသွေအချိုးရှိ ကွန်ပွဲများက အချိုးဆောင်ချပါ လည်း လုပ်ဆောင်ချခြင်းကို ဆောင်သည် ။

2. **စက်ရှင်းခြင်း ရှိသော ကွန်ပွဲများ**:
   - စက်ရှင်းခြင်း ရှိသော ကွန်ပွဲများက အချိုးဆောင်ချပါ လည်း လုပ်ဆောင်ချခြင်းကို ဆောင်သည် ။

3. **အချိုးဆောင်ခြင်း ရှိသော ကွန်ပွဲများ**:
   - အချိုးဆောင်ခြင်း ရ